## <span style=color:blue>Patterns used in Programming Assignment 2 (version mostly avoiding the util.py file)  </span>

In [23]:
# These are boiler plate imports that seem useful
# Perhaps cleaner would be to delete or comment out the ones that aren't used in this script...

import sys
import json
import csv
import yaml

import pandas as pd
import numpy as np

import datetime;

import matplotlib as mpl

import time
from datetime import datetime
# see https://stackoverflow.com/questions/415511/how-do-i-get-the-current-time-in-python
#   for some basics about datetime

import pprint

import os
from dotenv import load_dotenv

# sqlalchemy 2.0 documentation: https://www.sqlalchemy.org/
import psycopg2
from sqlalchemy import create_engine, text as sql_text

# the following is deprecated, it seems, so using the sqlalchemy
# from pyscopg2 import sqlio

# the file in benchmarking/util.py should hold utilities useful for your benchmarking exercise
# In this notebook we have commented out all mentions of util, so that you can run
#    this notebook before setting up your benchmarking/util.py file
# sys.path.append('benchmarking/')
# import util
# to invoke a function "foo()" inside util.py, use "util.foo()"

In [24]:
dotenv_path = 'variables.env'
load_dotenv(dotenv_path=dotenv_path)

True

In [25]:

load_dotenv()

schema = os.getenv('DISC_4_SCHEMA')
port = os.getenv('DISC_4_PORT')
host = os.getenv('DISC_4_HOST')
database = os.getenv('DISC_4_DB')
username = os.getenv('username')
password = os.getenv('password')

In [26]:
# following https://www.geeksforgeeks.org/connecting-postgresql-with-sqlalchemy-in-python/

db_eng = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}",
                       connect_args={'options': '-csearch_path={}'.format(f"{schema}")},
                       isolation_level = 'SERIALIZABLE')
#    , echo=True)
#    , echo_pool="debug")

print("Successfully created db engine.")

# connect_args is used to set search_path to the schema 'new_york_city' in the airbnb database

# isolation_level SERIALIZABLE makes transactions happen in sequence, which is good 
#      for the benchmarking we will be doing

# for general info on sqlalchemy connections,
#    see: https://docs.sqlalchemy.org/en/20/core/connections.html

# echo from https://docs.sqlalchemy.org/en/20/core/engines.html

Successfully created db engine.


In [27]:
sys.path.append('/Users/Nfaith21/Documents/ECS 116 - Misc/DISC_5_FILES/benchmarking/')
import util_main as util

In [28]:
new_dict = {}

years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
        '2020', '2021', '2022', '2023', '2024']
for year in years:
    new_dict['q_listings_join_reviews_'+year] = util.build_query_listings_join_reviews_datetime(year+'-01-01 12:00:00.000', year+'-12-31 12:00:00.000')


In [29]:
# Initialize an empty dictionary to store performance details for all queries
all_perf_details = {}

all_indexes = [['datetime','reviews'], ['id','listings']] 


# Define index combinations to test in the specified order
index_combinations = [
    [],
    [['datetime', 'reviews']],
    [['id', 'listings']],
    [['datetime', 'reviews'], ['id', 'listings']]
]

count = 50

# Iterate through each query and its associated data
for query_name, query in new_dict.items():
    print('Processing query:', query_name)

    # Initialize a dictionary to store performance details for the current query
    query_perf_details = {}

    # Iterate through each index combination in the specified order
    for spec in index_combinations:
        # Get the current index combination name using build_index_description_key function
        spec_name = util.build_index_description_key(all_indexes, spec) if spec else '__'

        print('Processing spec:', spec_name)

        # Drop indexes not in the current spec
        for index in all_indexes:
            if index not in spec:
                mod_index = util.add_drop_index(db_eng, 'drop', index[0], index[1])
                print(f'\nAfter dropping {index}, the indexes are:')
                print(mod_index)

        # Add indexes in the current spec
        for index in spec:
            mod_index = util.add_drop_index(db_eng, 'add', index[0], index[1])
            print(f'\nAfter adding {index}, the indexes are:')
            print(mod_index)

        # Calculate performance metrics for the current query and spec
        perf_details = util.calc_time_diff_per_year(db_eng, count, {query_name: query})

        # Add the performance details to the query's performance dictionary under the spec_name key
        query_perf_details[spec_name] = perf_details[query_name]

    # Add the query's performance details to the overall performance dictionary
    all_perf_details[query_name] = query_perf_details

# Write all performance data to JSON file
util.write_perf_data(all_perf_details, 'listings_join_reviews.json')


Processing query: q_listings_join_reviews_2009
Processing spec: __

After dropping ['datetime', 'reviews'], the indexes are:
[('houses', 'reviews', 'date_in_reviews', None, 'CREATE INDEX date_in_reviews ON houses.reviews USING btree (date)')]

After dropping ['id', 'listings'], the indexes are:
[]

The list of running times for q_listings_join_reviews_2009 is as follows:
[1.272872,
 1.029472,
 1.050999,
 1.009079,
 0.901929,
 0.799065,
 0.706871,
 0.688113,
 0.641502,
 0.514055,
 0.133668,
 0.078446,
 0.096309,
 0.077492,
 0.07667,
 0.075595,
 0.076613,
 0.076119,
 0.081425,
 0.078873,
 0.094307,
 0.075744,
 0.080519,
 0.076059,
 0.075382,
 0.09073,
 0.076021,
 0.076958,
 0.076802,
 0.093559,
 0.076476,
 0.076356,
 0.076164,
 0.091328,
 0.095198,
 0.145227,
 0.253342,
 0.106582,
 0.146129,
 0.08152,
 0.076294,
 0.077464,
 0.07729,
 0.077407,
 0.177311,
 0.077614,
 0.078266,
 0.077072,
 0.07968,
 0.102267]

The statistics on the list of running times for q_listings_join_reviews_2009 are

In [30]:
  q5 = """
SELECT CAST(EXTRACT(YEAR FROM datetime) AS INTEGER) AS year, COUNT(*) AS review_count
FROM reviews
group by year;
"""
with db_eng.connect() as conn:
    result_reviews = conn.execute(sql_text(q5))
    print()
    print(result_reviews.all())



[(2010, 449), (2024, 8710), (2023, 228831), (2016, 48527), (2009, 56), (2011, 1905), (2017, 66146), (2020, 51172), (2021, 109415), (2022, 196136), (2015, 28465), (2012, 3872), (2018, 95137), (2014, 14203), (2019, 126469), (2013, 7317)]
